# Tratamento de Datos Bank of Engrand

""" Create the data set from the raw data from "http://www.macrohistory.net/data/" according to the specifications in the Config object"""

# (1) Imports

In [95]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import sys
import statsmodels.api as sm
import scipy.stats as st
import os
from operator import itemgetter
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from scipy.stats import mstats
import statsmodels.formula.api as smf

pd.set_option('display.max_rows',None)

In [96]:
# we do not compute growth rates for the interest rates and the slope for the yield curve.
no_change = ["drate",  "global_drate", "lrate", "srate"]

horizon = 2  # Horizon of percentage and ratio changes (in years)
data_years_pre_crisis = 2  # number of years before a crisis for
                           # which outcome is set positive
data_include_crisis_year = False  # Whether to exclude the actual crisis
                                  # observation and only predict years a head of a crisis

predictors = ["drate", "cpi_pdiff" , "bmon_gdp_rdiff", "stock_pdiff",
              "cons_pdiff" ,"pdebt_gdp_rdiff", "inv_gdp_rdiff", "ca_gdp_rdiff",
              "tloan_gdp_rdiff","tdbtserv_gdp_rdiff", "global_loan"]
# For the other predictors we compute growth rate (percentage change or ratio change)
# and add the horizon (e.g. 2 year change) to the variable name

predictors = [p + str(horizon) for p in predictors if p not in no_change] +\
list(set(predictors).intersection(set(no_change)))

In [97]:
# Dataset_Bruto_Original
#df_jst = pd.read_excel('/content/sample_data/Bruto_Original.xlsx')
# Dataset Mean
#df_jst = pd.read_excel('/content/sample_data/Mean.xlsx') 
# Dataset Median
#df_jst = pd.read_excel('/content/sample_data/Median.xlsx') 
# Dataset Most_F
#df_jst = pd.read_excel('/content/sample_data/Most_F.xlsx') 

In [98]:
df = df_jst.copy()
df_jst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rconpc       2352 non-null   float64
 1   gdp          2352 non-null   float64
 2   iy           2352 non-null   float64
 3   ca           2352 non-null   float64
 4   imports      2352 non-null   float64
 5   exports      2352 non-null   float64
 6   narrowm      2352 non-null   float64
 7   money        2352 non-null   float64
 8   stir         2352 non-null   float64
 9   ltrate       2352 non-null   float64
 10  stocks       2352 non-null   float64
 11  debtgdp      2352 non-null   float64
 12  revenue      2352 non-null   float64
 13  expenditure  2352 non-null   float64
 14  xrusd        2352 non-null   float64
 15  tloans       2352 non-null   float64
 16  tmort        2352 non-null   float64
 17  thh          2352 non-null   float64
 18  tbus         2352 non-null   float64
 19  hpnom 

In [99]:
# rename variables
df.rename(columns={"crisisJST": "crisis",'stir': 'srate','ltrate': 'lrate','iy': 'inv_gdp',
                   'debtgdp': 'pdebt_gdp','money': 'bmon','narrowm': 'nmon','tloans': 'tloan','tbus': 'bloan',
                   'thh': 'hloan','tmort': 'mort','stocks': 'stock','hpnom': 'hp','rconpc': 'cons'}, inplace=True)

In [100]:
#exclude periods that are not normal economic conditions (e.g. WW2)

exclude_ix = np.zeros(len(df)) > 1

data_exclude_extreme_period = True  # Whether to exclude WW1, WW2 and 
data_period = 'all'  # The time frame investigate. Either 'all' observations,
        # or 'pre-ww2' or 'post-ww2'.
if data_exclude_extreme_period:
  # exclude great depression | but NOT the beginnig of this crisis
  exclude_ix = exclude_ix | (np.array(df["year"] > 1933) & np.array(df["year"] < 1939))
  # exclude WW1
  exclude_ix = exclude_ix | (np.array(df["year"] > 1913) & np.array(df["year"] < 1919))
  # exclude WW2
  exclude_ix = exclude_ix | (np.array(df["year"] > 1938) & np.array(df["year"] < 1946))

if not data_period in ['all', 'pre-ww2', 'post-ww2']:
  raise ValueError("time split is either 'all', 'pre-ww2', or 'post-ww2'")
elif data_period  == 'pre-ww2':
  exclude_ix = exclude_ix | np.array(df["year"] > 1939)
elif data_period == 'post-ww2':
  exclude_ix = exclude_ix | np.array(df["year"] < 1946)

feature_names = set(df.columns.values).difference(set(['year', 'country', 'iso', 'crisis_id','crisis']))
# set all feature values to NA in the excluded periods
df.loc[exclude_ix, feature_names] = np.nan


In [101]:
df.loc[:, 'drate'] = df['lrate'] - df['srate']  # rate differential
df.loc[:, 'pdebt'] = df['pdebt_gdp'] * df['gdp'] # compute public debt from public debt/gdp ratio
df.loc[:, 'inv'] = df['inv_gdp'] * df['gdp']  # compute investment from investment/gdp ratio

In [102]:
# Calculaute debt to service ratios
df.loc[:, 'tdbtserv'] = df['tloan'] * df['lrate'] / 100.0
    
pre_gdp_ratios = ['bmon', 'nmon', 'tloan', 'bloan',
                  'hloan', 'mort', 'ca', 'cpi', 'tdbtserv',
                  'inv', 'pdebt'] # vector of variables that will be transformed by GDP ratio

def make_ratio(data_input, variables, denominator="gdp"):
    """ Computes the ratio of two variables. By detault the denominator is GDP. """

    names_out = []
    if isinstance(variables, str):
        variables = [variables]
    data = data_input.copy()
    for var in variables:
        varname = var + '_' + denominator
        data[varname] = data[var] / data[denominator]
        names_out.append(varname)
    return data, names_out

df, gdp_ratios = make_ratio(df, pre_gdp_ratios, denominator='gdp')
display(df)

Output hidden; open in https://colab.research.google.com to view.

In [103]:
# display(df)
# here we compute the transformations and att the variables to the dataset df 
# ratio change by GDP (rdiff)

def lag_pct_change(x, h):
    """ Computes percentage changes """
    lag = np.array(pd.Series(x).shift(h))
    return (x - lag) / lag

def make_shift(data_input, variables, type, horizon=5):
    """ Computes the change of a variable with respect to a certain horizon. """
    
    names_out = []
    data = data_input.copy()
    data_group = data.groupby('iso')
    if isinstance(variables, str):
        variables = [variables ]
    for var in variables:
        if type == "absolute":
            varname = var + '_rdiff' + str(horizon)
            data[varname] = data_group[var].diff(horizon)
        elif type == "percentage":
            varname = var + '_pdiff' + str(horizon)
            # attention objects must be ordered by year and country as they are in the original data
            data[varname] = data_group[var].apply(lambda x: lag_pct_change(x, h=horizon))
            #data[varname] = data_group[var].pct_change(horizon)

        names_out.append(varname)
    return data, names_out


# ratio change by GDP (rdiff)
df, _ = make_shift(df, ["lrate", "srate", "drate"] + gdp_ratios,
                   type="absolute",horizon=horizon)
# percentage change (pdiff)
df, _ = make_shift(df, ['stock', 'cpi', 'hp', 'cons', 'gdp'] + pre_gdp_ratios,
                   type="percentage",horizon=horizon)  # do not use absolute change

display(df)



Output hidden; open in https://colab.research.google.com to view.

In [104]:
def hamilton_filter(group, col, h=2, p=4, output="cycle"):  
    """ computes Hamilton filter
    : param int h: look-head period
    : param int p: number of lagged variables
    """

    x = group[col].values
    # note: Hamilton used 100 times x's logrithm in his employment data,
    # however, this is commented out because our data has negative values
    # x = 100*np.log(x)
    # Get the trend/predicted series
    trend = hamilton_filter_glm(x, h, p)
    if trend is not None:  # if dataframe passed is not full of nans
        # Get the cycle which is simple the original series substracted by the trend
        cycle = x - trend
        # Get the random walk series which is simply the difference between
        # the original series and the h look back
        df_x = pd.DataFrame(x)
        df_x_h = df_x.shift(h)
        random = df_x - df_x_h
    else:
        trend = x
        cycle = x
        random = x
    # Return required series in result, if all is selected then all results
    # are returned in a data frame
    if (output == "x"):
        return x
    elif (output == "trend"):
        return trend
    elif (output == "cycle"):
        return np.asarray(cycle)
    elif (output == "random"):
        return random
    elif (output == "all"):
        df = pd.DataFrame()
        df['x'] = x
        df['trend'] = trend
        df['cycle'] = cycle
        df['random'] = random
        df.plot()
        # pyplot.show()
        return df
    else:
        print ('\nInvalid output type')



def hamilton_filter_glm(x, h=2, p=4):
    """ Runs the linear model for the specification of the hamilton filter """
    # Create dataframe for time series to be smoothed, the independent variable y
    df = pd.DataFrame(x)
    df.columns = ['yt8']
    # Create matrix of dependent variables X which are the shifts of 8 period back
    # for 4 consecutive lags on current time t
    for lag in range(h, (h + p)):
        df['xt_' + str(lag - h + 1)] = df.yt8.shift(lag)
    # Getting the dependent varaibles X's index names
    X_columns = []
    for i in range(1, p + 1):
        new_s = 'xt_' + str(i)
        X_columns.append(new_s)
    # y and X variables for regression
    y = df['yt8']
    X = df[X_columns]

    xt_0 = pd.DataFrame(np.ones((df.shape[0], 1)))
    xt_0.columns = ['xt_0']
    X = xt_0.join(X)
    # Build the OLS regression model and drop the NaN
    try:
        if (sum(np.isnan(y)) != y.size):
            model = sm.OLS(y, X, missing='drop').fit()
            # Optional: print out the statistics
            model.summary()
            predictions = model.predict(X)
            return predictions
        else:
            return y
    except ValueError:
        pass

def all_same(items):
    return all(x==items[0] for x in items)

def sigmoid(x):
  return 1/(1 + np.exp(-x))

def sigmoidinv(x):
  return -np.log(1.0/x -1)

def normalize(data):
  return data.apply(normalizeV)

def normalizeV(x):
  x = x.astype(dtype="float32")
  return (x- np.nanmin(x))/(np.nanmax(x) - np.nanmin(x))


def make_level_change(data_input, variables, type, horizon=10):
    """ Computes the hamilton filter or difference from moving average
     :param pd.dDtaFrame data_input: Dataset. The tranformed variable will be appended to that data
     :param list of str variables: Name of the variables in data_input that will be transformed
     :param str type: Type of transformation. Either "ham" (hamilton filter) or "mad" (movgin average difference).
    """
    names_out = []
    data = data_input.copy()
    data_grouped = data.groupby('iso')
    if isinstance(variables, str):
        variables = [variables]
    for var in variables:
        if type == "mad":
            varname = var + '_mad'
            data[varname] = np.nan
            data_mad = pd.DataFrame(data_grouped.apply(mov_ave_diff, var, horizon), 
                                    columns=[varname])
            for iso in data_mad.index.values:
                data.loc[data.iso == iso, varname] = data_mad.loc[iso, varname]

        if type == "ham":
            varname = var + '_ham'
            data[varname] = np.nan
            data_ham = pd.DataFrame(data_grouped.apply(hamilton_filter, var, 2, 4),
                                    columns=[varname])
            for iso in data_ham.index.values:
                data.loc[data.iso == iso, varname] = data_ham.loc[iso, varname]
        names_out.append(varname)
    return data, names_out
    
# hamilton filter (ham)
df, _ = make_level_change(df, ["cons"] + gdp_ratios, type="ham")

In [105]:
# --- Computing global variables --- #

# global credit growth (global_loan)
for year in df["year"].unique():
  ix = df["year"] == year
  for country in df["iso"].unique():
  # computing the average across all countries but the selected one
    perc_pos = df.loc[ix.values & (df.iso != country).values,"tloan_gdp_rdiff" + str(horizon)].mean()
    if not np.isnan(perc_pos):
      df.loc[ix.values & (df.iso == country).values,"global_loan" + str(horizon)] = perc_pos

In [106]:
# global slope of the yield curve
for year in df["year"].unique():
  ix = df["year"] == year
  for country in df["iso"].unique():
    # computing the average across all countries but the selected one
    perc_pos = df.loc[ix.values & (df.iso != country).values, "drate"].mean()
    if not np.isnan(perc_pos):
      df.loc[ix.values & (df.iso == country).values, "global_drate"] = perc_pos

In [107]:
# Names of the indicators used as predictors
# check whether we have created all features that will be used in the experiment
if len(set(predictors).difference(set(df.columns.values))) > 0:
  raise ValueError('Features ' + ', '.join(set(predictors).difference(set(df.columns.values))) + "\n" +"could not be found in the data!")

In [108]:
# --- creating the 'landing zone' on the crisis outcome --- #
years = df.year.values
isos = df['iso'].values
# number of years before a crisis for
# which outcome is set positive

crisis_in = df_jst.crisisJST.values == 1
crisis = crisis_in * 0
for i, (yr, cr) in enumerate(zip(years, crisis_in)):
  if cr:
    for l in np.arange(1, 1 + data_years_pre_crisis): # flagging years before crisis as positive 
      if yr > (np.min(years) + l - 1):
        crisis[i - l] = 1
    if data_include_crisis_year: 
      crisis[i] = 1  # crisis year

In [109]:
data_post_crisis = 4  #  How many observations (in years) after the 
                      # crisis should be deleted to avoid post-crisis bias

# --- treatment of actual crisis and post crisis observations --- #
i_keep = np.ones(len(df), dtype=int)
for i, (yr, cr, iso) in enumerate(zip(years, crisis_in, df.iso)):
  if cr:
    if not data_include_crisis_year: 
      i_keep[i] = 0
    for j in range(1, 1 + data_post_crisis):
      k = i + j
      if (iso == df.iso[k]) & (k < len(df)):
        i_keep[k] = 0

In [110]:
# --- Give all observations of the same crisis the same ID --- #
# This ID is used for cross-validation to make sure that the same crisis
# is not in the training and test set
# This function generalizes to any length of crises

crisis_id = np.zeros(len(df))
count = int(1)
for i in np.arange(2, len(df)):
  if crisis[i] == 1:
    if not ((crisis[i - 1] == 1) & (isos[i] == isos[i - 1])):
      count += 1
      crisis_id[i] = count

In [111]:
# All other observations get unique identifier
crisis_id[crisis_id == 0] = np.random.choice(sum(crisis_id == 0),
                                             size=sum(crisis_id == 0),
                                             replace=False) + 2 + int(max(crisis_id))

In [112]:
# create the data set
features = df.loc[:, predictors]
data = features
data['crisis'] = crisis.astype(int)
data['crisis_id'] = crisis_id.astype(int)
data['year'] = years.astype(int)
data['iso'] = isos # name of countries

exclude_ix = exclude_ix | (i_keep == 0)
data = data.loc[~exclude_ix, :]
data = data.dropna()  # remove missing values
data = data.reset_index(drop=True)  # update index

In [113]:
data.info()
display(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1620 entries, 0 to 1619
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   cpi_pdiff2           1620 non-null   float64
 1   bmon_gdp_rdiff2      1620 non-null   float64
 2   stock_pdiff2         1620 non-null   float64
 3   cons_pdiff2          1620 non-null   float64
 4   pdebt_gdp_rdiff2     1620 non-null   float64
 5   inv_gdp_rdiff2       1620 non-null   float64
 6   ca_gdp_rdiff2        1620 non-null   float64
 7   tloan_gdp_rdiff2     1620 non-null   float64
 8   tdbtserv_gdp_rdiff2  1620 non-null   float64
 9   global_loan2         1620 non-null   float64
 10  drate                1620 non-null   float64
 11  crisis               1620 non-null   int64  
 12  crisis_id            1620 non-null   int64  
 13  year                 1620 non-null   int64  
 14  iso                  1620 non-null   object 
dtypes: float64(11), int64(3), object(1)
me

,cpi_pdiff2,bmon_gdp_rdiff2,stock_pdiff2,cons_pdiff2,pdebt_gdp_rdiff2,inv_gdp_rdiff2,ca_gdp_rdiff2,tloan_gdp_rdiff2,tdbtserv_gdp_rdiff2,global_loan2,drate,crisis,crisis_id,year,iso
0,-6.153822e-02,0.041149,6.268976e-02,-0.017004,-1.764800e-02,2.117240e-02,6.404354e-02,-0.016960,-1.261305e-03,-0.006337,0.137350,0,1169,1872,AUS
1,-4.687499e-02,-0.004738,3.362203e-01,0.166782,-4.910700e-02,2.040710e-02,-6.631564e-02,-0.008764,-8.481443e-04,-0.004611,0.271958,0,448,1873,AUS
2,4.918032e-02,-0.025482,2.200121e-01,0.112547,3.940200e-02,1.152190e-02,-5.394574e-02,0.003071,-6.339717e-05,0.012690,0.153317,0,342,1874,AUS
3,8.196707e-02,0.017768,1.051117e-01,0.103832,9.099500e-02,3.557780e-02,1.515147e-02,0.020566,5.231474e-04,0.025359,-0.092675,0,273,1875,AUS
4,4.687499e-02,0.032032,5.342820e-02,0.057469,-3.581600e-02,2.591790e-02,-2.987871e-03,0.028408,1.079837e-03,0.015678,-0.034092,0,2316,1876,AUS
5,4.545455e-02,0.042715,5.467626e-02,-0.017626,-5.759500e-02,4.936960e-02,-2.032869e-02,0.056262,2.154277e-03,0.014139,-0.111150,0,659,1877,AUS
6,1.492513e-02,0.019693,1.430615e-02,0.070724,1.434600e-02,1.326320e-02,-2.803749e-02,0.051898,1.964759e-03,0.032348,-0.357200,0,2244,1878,AUS
7,-4.347826e-02,-0.013164,-1.705321e-01,0.051629,3.284200e-02,-4.842500e-02,9.750688e-03,-0.014961,2.227251e-06,0.031042,-0.296650,0,1774,1879,AUS
8,-5.882330e-02,0.005936,-5.218618e-02,-0.033548,4.936000e-02,-4.668600e-03,5.265502e-02,-0.055831,-2.571624e-03,0.001913,-0.290592,0,1846,1880,AUS
9,-4.545455e-02,0.045504,3.042763e-01,0.047876,2.326800e-02,5.083690e-02,-2.058774e-02,0.012008,-2.070286e-03,0.002156,-0.918500,0,449,1881,AUS


In [114]:
"""#create excel file

df_boe = pd.DataFrame(data)
df_boe.to_excel(r'/content/drive/MyDrive/Bank_ready.xlsx', index = False)"""


"#create excel file\n\ndf_boe = pd.DataFrame(data)\ndf_boe.to_excel(r'/content/drive/MyDrive/Bank_ready.xlsx', index = False)"

In [115]:
"""#create excel file

df_mean = pd.DataFrame(data)
df_mean.to_excel(r'/content/drive/MyDrive/Mean_ready.xlsx', index = False)"""

"#create excel file\n\ndf_mean = pd.DataFrame(data)\ndf_mean.to_excel(r'/content/drive/MyDrive/Mean_ready.xlsx', index = False)"

In [116]:
"""#create excel file

df_median = pd.DataFrame(data)
df_median.to_excel(r'/content/drive/MyDrive/Median_ready.xlsx', index = False)"""

"#create excel file\n\ndf_median = pd.DataFrame(data)\ndf_median.to_excel(r'/content/drive/MyDrive/Median_ready.xlsx', index = False)"

In [117]:
"""#create excel file

df_most_f = pd.DataFrame(data)
df_most_f.to_excel(r'/content/drive/MyDrive/Most_f_ready.xlsx', index = False)"""